# Machine Learning part
- Build a ML model to explain the rank
- Topic: Rank Order of Candidates for Heart Transplantation in France: An Explainable Machine Learning analysis
- Authors : Martin Prodel (MS, PhD), Benoit Audry (MS)
- Created in 2025 

# Imports

## import packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GroupKFold, GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pickle
import shap
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
from matplotlib.colors import ListedColormap
import matplotlib.ticker as ticker

# internal
from ml_training_ABM import train_cv_regressor, train_cv_regressor_group_effect
from ml_data_viz_ABM import (
    train_vs_test_error_curve,
    residual_plot,
    histo_residuals,
    feature_importance,
    predicted_vs_actual,
    learning_cure,
    partial_dependence_plot,
    compute_shap_values,
    compute_shap_interaction_values,
    decision_path_analysis,
    trend_threshold_detection,
    draw_single_shap_plot,
    draw_single_shap_plot_uni,
)

from params_ABM import (
    target_col,
    variables_deduced,
    variables_donneur,
    variables_redundant_sup0,
    map_col_name,
)

from benchmark_ml_ABM import (
    run_benchmark,
    models_bench,
    params_regressors,
    show_results_bench,
)

import jupyter_black

jupyter_black.load()

%load_ext autoreload
%autoreload 2


## load data

In [ ]:
# Importation depuis un fichier pickle
df = pd.read_pickle("../data/df_for_ml_article.pkl")

In [ ]:
df = df[~df["IDD"].isin(IDD_one_row)]

## Slight preprocessing 
- It was debated by the scientific commitee afterward, during the machine learning phase of the project)

In [ ]:
df["DecilePEPT_IMP"] = np.where(df["DecilePEPT"].isna(), 5, df["DecilePEPT"])
df["DecilePEPT_IMP"] = np.where(df["CEC2"] == "O", np.nan, df["DecilePEPT_IMP"])
print(df.DecilePEPT_IMP.value_counts())
print(df.DecilePEPT_IMP.isna().sum())

In [ ]:
df = df[
    [
        col
        for col in df.columns
        if col
        not in variables_deduced
        + variables_redundant_sup0
        + ["SCD", "DecilePEPT", "CEC2"]
    ]
]

df.head()
nb_rows_full_df = len(df)
df.info(memory_usage="deep")

In [ ]:
# Remove certain outliers (< 10 data points), impossible vital parameters values
df.DFG = df.DFG.clip(upper=150)
df.DFG_AVI = df.DFG_AVI.clip(upper=150)

In [ ]:
# Rename columns
for col in df.columns:
    if col not in map_col_name:
        print(col)
    elif col != target_col:
        map_col_name[col] = map_col_name[col] + f" ({col})"

df.columns = [map_col_name[col] for col in df.columns]

In [ ]:
# You may hade '.head(X)' to only use the X first rows of the df
df = df.sort_values(map_col_name["IDD"])  # .head(5000)

# Train / Test data

In [ ]:
# Group effect: a stratification strategy was used to avoid data leakage during the training process
# The data split was performed at the donor level, which resulted in candidate-donor pairs generated from the same donor
# being assigned to the same subset. This stratification was required to avoid data leakage (Kaufman S leakage in data mining:
# formulation, detection and avoidance, ACM Transactions on Knowledge Discovery from Data 2012) since multiple candidates
# shared the same donor characteristics. If the split had been made across training and test sets, the model could have learned
#  donor-specific patterns during training, and leveraged this information in the testing samples, leading to over-optimistic
#  performance estimates.

activate_group_mode = True

## Split train/test/weight/group

In [ ]:
print(df[map_col_name["ALLOC"]].value_counts())
df[map_col_name["ALLOC"]] = df[map_col_name["ALLOC"]].replace(
    {"ADUSTAND": "ADULTSTAND"}
)
print(df[map_col_name["ALLOC"]].value_counts())

In [ ]:
# Same weight for all observations
df[map_col_name["observation_weight"]].value_counts()
df[map_col_name["observation_weight"]] = 1
df[map_col_name["observation_weight"]].value_counts()

In [ ]:
# Define relevant columns
weight_col = map_col_name["observation_weight"]
group_col = map_col_name["IDD"]

# Verify that the necessary columns exist
if target_col not in df.columns or weight_col not in df.columns:
    raise ValueError(
        f"The columns '{target_col}' or '{weight_col}' are missing from the DataFrame."
    )

# Clean non-numeric columns
X = df.drop(columns=[target_col, weight_col, group_col, map_col_name["IDR"]])

# Encode categorical variables
# In the future, one might think other ways to do better than this method ? (try other approaches)
X = pd.get_dummies(X, drop_first=True)

# Séparation des variables explicatives et cible
y = df[target_col]

# Pondérations
weights = df[weight_col]

# Effets groupes (mesures répétées pour un même patient, séparer les groupes par ID de patient)
groups = df[group_col]
gkf = GroupKFold(n_splits=5)  # a CV using the groups

In [ ]:
# ⚠️ If we IGNORE the group effect (repeated measures) - this is not RECOMMENDED !
if not activate_group_mode:
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
        X, y, weights, test_size=0.2, random_state=42
    )

In [ ]:
# # ⚠️ Use stratification

# 🚀 Create the training and test datasets by separating the groups
if activate_group_mode:
    # Use GroupShuffleSplit to split by groups (to avoid data leakage between train and test)
    # This ensures that all samples from the same group are either in the train set or in the test set,
    # which is crucial for models that account for group effects.

    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_idx, test_idx = next(gss.split(X, y, groups))

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    weights_train, weights_test = weights.iloc[train_idx], weights.iloc[test_idx]
    groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]

    print("We are ready for machine learning with group effects :)")

    print("Intersection:", set(groups_train) & set(groups_test))  # ca doit etre vide !!

# XGboost

In [ ]:
# create an XGBoost regression model
from params_ABM import xgboosst_grid

regressor = XGBRegressor(
    n_estimators=1000,
    max_depth=7,  # for each tree
    eta=0.1,  # learning rate
    subsample=0.7,
    colsample_bytree=0.8,  # subsampling of columns
)

if activate_group_mode:
    # Training CV on the grid
    y_pred, best_model = train_cv_regressor_group_effect(
        X_train,
        y_train,
        X_test,
        y_test,
        weights_train,
        weights_test,
        groups_train,
        regressor,
        xgboosst_grid,
        gkf,
    )
    print("training complete (with group effect)")
else:
    # Training CV on the grid
    y_pred, best_model = train_cv_regressor(
        X_train,
        y_train,
        X_test,
        y_test,
        weights_train,
        weights_test,
        regressor,
        xgboosst_grid,
    )

In [ ]:
# ⚠️ Quite long to run
partial_dependence_plot(best_model, X)

In [ ]:
train_vs_test_error_curve(
    X_train, y_train, X_test, y_test, weights_train, model_type="xgb"
)

In [ ]:
residuals = residual_plot(y_test, y_pred)

In [ ]:
histo_residuals(residuals)

In [ ]:
feature_importance(best_model, X)

In [ ]:
# Translate labels from French to English
translate_dict = {
    "Temps relatif du donneur (TimelineD)": "Rank by study period",
    "Age du donneur (AGED)": "Donor age",
    "Sexe donneur (SEXD)": "Donor sex",
    "Taille donneur (TAILLED)": "Donor height",
    "Poids donneur (POIDSD)": "Donor weight",
    "Index points bonus assistance gauche (+750 pts) (IndALD)": "Provision to stable LVAD adult candidates",
    "Age receveur (AGER)": "Candidate age",
    "Sexe receveur (SEXER)": "Candidate sex",
    "Taille receveur (TAILLER)": "Candidate height",
    "Poids receveur (POIDSR)": "Candidate weight",
    "Délai d'inscription (DelaiINSC)": "Waiting list duration",
    "Délai CEC2 receveur (en j) (DelaiCEC2)": "VA-ECMO support duration",
    "Délai ass. ventriculaire (DelaiAV2)": "LVAD support duration",
    "Bilirubine receveur (µmol/L) (BILI2)": "Total bilirubin",
    "Bilirubine avant CEC2 ou DRG2 (BILI_AVI)": "Bilirubin before initiation of VA-ECMO or inotropes",
    "Décile Peptides avant CEC2 ou DRG2 (1 à 10) (DecilePEPT_AVI)": "Natriuretic peptide before initiation of VA-ECMO or inotropic infusion",
    "Distance (en minutes) (DIST)": "Geographic component",
    "Débit de filtration glomérulaire (DFG)": "eGFR",
    "Débit de filtration glomérulaire avant CEC2 ou DRG2 (DFG_AVI)": "eGFR before initiation of VA-ECMO or inotropes",
    "Décile Peptides (DecilePEPT_IMP)": "Natriuretic peptide / VA-ECMO",
    "Groupe sanguin donneur (ABOD)_AB": "Blood type AB donor",
    "Groupe sanguin donneur (ABOD)_B": "Blood type B donor",
    "Groupe sanguin donneur (ABOD)_O": "Blood type 0 donor",
    "Groupe sanguin receveur (ABOR)_AB": "Blood type AB candidate",
    "Groupe sanguin receveur (ABOR)_B": "Blood type B candidate",
    "Groupe sanguin receveur (ABOR)_O": "Blood type 0 candidate",
    "Drogues inotropes receveur (DRG2)_O": "Inotropes",
    "Type d'assistance ventriculaire (SIAV2)_G": "Uncomplicated LVAD",
    "Composante score (adulte/pédiatrie +/- urgence) (ALLOC)_ADULTURG": "Exceptions in adult candidates without durable MCS",
    "Composante score (adulte/pédiatrie +/- urgence) (ALLOC)_ADULTURG_AV": "Exception in adult candidates with LVAD-related complication",
    "Composante score (adulte/pédiatrie +/- urgence) (ALLOC)_PEDSTAND": "Non-urgent pediatric candidates",
    "Composante score (adulte/pédiatrie +/- urgence) (ALLOC)_PEDURG": "Urgent pediatric candidates",
    "Maladie receveur (MAL)_Dilated cardiomyopathy": "Dilated cardiomyopathy as indication",
    "Maladie receveur (MAL)_Other": "Other indication for transplantation",
    "Maladie receveur (MAL)_Valvular or Congenital heart disease": "Valvular or congenital cardiomyopthy as indication",
    "Cœur Artificiel Total (ou BV) (CAT_BV)_O": "BiVAD or TAH",
}
X_lab = X.rename(columns=translate_dict)

In [ ]:
feature_importance(best_model, X_lab)

In [ ]:
predicted_vs_actual(y_test, y_pred)

In [ ]:
learning_cure(best_model, X, y, groups)  # train the model for 5 sizes of data samples

In [ ]:
print("Data viz terminée.")

# XAI
- https://christophm.github.io/interpretable-ml-book/pdp.html

Key Notes for Decision Trees and Random Forests
1. SHAP Values for Trees
Tree-based models like Decision Trees and Random Forests are fully supported by TreeExplainer.
SHAP is computationally efficient because it uses tree-specific optimizations.
2. Feature Importance
SHAP values provide a better and more consistent feature importance measure compared to built-in importance measures (like feature_importances_).
3. Interpretability
Use summary_plot to see the global importance of features.
Use dependence_plot to analyze the effect of a specific feature on predictions.
Use force_plot to explain individual predictions.
4. For Random Forests
SHAP computes values for the ensemble of trees in the forest, considering the contribution of each tree in the ensemble.

*Example Outputs*   
SHAP Summary Plot:
- Shows the importance of each feature and its impact on the model’s predictions.   

SHAP Dependence Plot:
- Visualizes how a specific feature contributes to the prediction.   

SHAP Force Plot:
- Explains a single prediction with feature contributions.


In [ ]:
# SHAP (classic) ~20min for XGboost
explainer, shap_values, explaination = compute_shap_values(
    best_model, X_test, max_nb_feature_show=30
)

In [ ]:
X_test_lab = X_test.rename(columns=translate_dict)

In [ ]:
# BEESWARM - Visualize SHAP values - Summary plot for feature importance

figsize = (18, 20)  # Figure size (width, height)
fig = plt.figure(figsize=figsize)
ax = fig.gca()
shap.summary_plot(
    shap_values,
    X_test_lab,
    max_display=40,  # Number of variables to display
    plot_size=(18, 20),
    cmap=plt.get_cmap("coolwarm"),  # Colors
    alpha=1,  # Transparency of points, useful for a large dataset like ours
    title="SHAP Summary Plot for XGBoost Model",
)
# ax.set_yticks(np.arange(0.5, 18.5, step=0.5))
ax.yaxis.set_major_locator(ticker.IndexLocator(base=0.5, offset=0))
plt.show()

In [ ]:
figsize = (18, 20)  # Figure size (width, height)
fig = plt.figure(figsize=figsize)
ax = fig.gca()
shap.plots.bar(
    explaination,
    max_display=40,
    show=False,
)
ax.set_yticks(
    np.arange(1, 37),
    labels=[
        "Blood type AB donor",
        "Valvular or congenital cardiomyopthy as indication",
        "Natriuretic peptide before initiation of VA-ECMO or inotropic infusion",
        "Dilated cardiomyopathy as indication",
        "BiVAD or TAH",
        "Candidate sex",
        "Blood type AB candidate",
        "Other indication for transplantation",
        "Urgent pediatric candidates",
        "Blood type B candidate",
        "Blood type 0 candidate",
        "eGFR before initiation of VA-ECMO or inotropes",
        "Blood type B donor",
        "Donor sex",
        "Non-urgent pediatric candidates",
        "VA-ECMO support duration",
        "Inotropes",
        "Bilirubin before initiation of VA-ECMO or inotropes",
        "Candidate height",
        "Candidate weight",
        "Waiting list duration",
        "Donor height",
        "Donor weight",
        "Blood type 0 donor",
        "Uncomplicated LVAD",
        "LVAD support duration",
        "Rank by study period",
        "Exception in adult candidates with LVAD-related complication",
        "Provision to stable LVAD adult candidates",
        "Exceptions in adult candidates without durable MCS",
        "Donor age",
        "eGFR",
        "Candidate age",
        "Geographic component",
        "Total bilirubin",
        "Natriuretic peptide / VA-ECMO",
    ],
    color="black",
)
plt.show()

In [ ]:
draw_single_shap_plot_uni(
    "VA-ECMO support duration",
    shap_values,
    X_test_lab,
    (-0.5, 18.5),
)

In [ ]:
draw_single_shap_plot_uni(
    "Natriuretic peptide / VA-ECMO",
    shap_values,
    X_test_lab,
    (99,),
    (-0.4, 0.6),
)

In [ ]:
draw_single_shap_plot_uni(
    "Exceptions in adult candidates without durable MCS",
    shap_values,
    X_test_lab,
    (99,),
    (-0.4, 0.7),
)

In [ ]:
txt_alloc = "Composante score (adulte/pédiatrie +/- urgence) (ALLOC)_ADULTURG_AV"
shap_URG_AV = explaination[:, txt_alloc].values[explaination[:, txt_alloc].data == True]
print(shap_URG_AV.size)
print(np.repeat(3, 5))

In [ ]:
data_Index_URG_AV = explaination[
    :, "Index points bonus assistance gauche (+750 pts) (IndALD)"
].data[explaination[:, txt_alloc].data == True]
print(data_Index_URG_AV)
print(data_Index_URG_AV.size)

In [ ]:
shap_AV = explaination[:, "Type d'assistance ventriculaire (SIAV2)_G"].values
data_AV = explaination[:, "Type d'assistance ventriculaire (SIAV2)_G"].data
data_Index_AV = explaination[
    :, "Index points bonus assistance gauche (+750 pts) (IndALD)"
].data
data_AV = data_AV * 1

In [ ]:
shap_AV_glob = np.concatenate((shap_AV, shap_URG_AV))
print(shap_AV_glob.shape)
data_AV_glob = np.concatenate((data_AV, np.repeat(2, shap_URG_AV.size)))
print(data_AV_glob.shape)
col_Index_AV_glob = np.concatenate((data_Index_AV, data_Index_URG_AV))
print(col_Index_AV_glob.shape)

In [ ]:
def jitter(values, amount):
    return values + np.random.uniform(-amount, amount, len(values))

In [ ]:
data_AV_glob_jitt = jitter(data_AV_glob, 0.3)

In [ ]:
fig, ax = plt.subplots()
scatter = ax.scatter(
    data_AV_glob_jitt,
    shap_AV_glob,
    c=col_Index_AV_glob,
    # cmap=ListedColormap(["deepskyblue", "orchid"]),
    cmap=ListedColormap(["#B24745FF", "#00A1D5FF"]),
    s=7,
)
fig.set_figheight(5)
fig.set_figwidth(10)
ax.set_xlabel("LVAD")
ax.set_ylabel("SHAP value for\n LVAD")
ax.set_title("SHAP Dependence Plot")
ax.set_xticks(
    [0, 1, 2],
    labels=[
        "No LVAD",
        "Uncomplicated LVAD",
        "Exception in adult candidates \nwith LVAD-related complication",
    ],
)
ax.set_ylim(-0.4, 0.6)
ax.legend(
    handles=scatter.legend_elements()[0],
    labels=["No", "Yes"],
    loc="upper left",
    title="Provision to stable LVAD adult candidates",
    alignment="left",
)
ax.axhline(0, color="gray", linestyle="--", linewidth=1)
ax.spines[["right", "top"]].set_visible(False)

In [ ]:
draw_single_shap_plot(
    "Candidate age",
    "Donor age",
    shap_values,
    X_test_lab,
    (99,),
)

In [ ]:
draw_single_shap_plot_uni(
    "Total bilirubin",
    shap_values,
    X_test_lab,
    (0, 160),
    (-0.5, 0.5),
)

In [ ]:
draw_single_shap_plot_uni(
    "Geographic component",
    shap_values,
    X_test_lab,
    (0, 300),
)

In [ ]:
draw_single_shap_plot_uni(
    "eGFR",
    shap_values,
    X_test_lab,
    (99,),
    (-0.5, 0.5),
)

In [ ]:
draw_single_shap_plot_uni(
    "Inotropes",
    shap_values,
    X_test_lab,
    (99,),
)

In [ ]:
figsize = (10, 5)  # Figure size (width, height)
fig = plt.figure(figsize=figsize)
ax = fig.gca()
ax.axhline(0, color="gray", linestyle="--", linewidth=1)
shap.dependence_plot(
    "Candidate sex",
    shap_values,
    X_test_lab,
    interaction_index=None,
    dot_size=16,
    alpha=0.9,
    x_jitter=0.3,
    title="SHAP Dependence Plot",
    ax=ax,
    show=False,  # Do not show the plot yet, we will adjust it
)
ax.set_xticks([1, 2], labels=["Male", "Female"])
ax.set_ylim(-0.4, 0.4)
plt.show()

In [ ]:
bench = run_benchmark(
    models_bench,
    params_regressors,
    X_train,
    X_test,
    y_train,
    y_test,
    weights_train,
    weights_test,
    groups_train,
    activate_group_mode,
    gkf,
)

In [ ]:
show_results_bench(bench, y_test)